# UCLA-Mono Lake Model (UCLA-MLM)

- #### This notebook provides non-python users access to a user interface of the UCLA-MLM.

- #### The notebook steps through the creation of export criteria, which is used with the UCLA-MLM Water Budget Model of Mono Lake.

- #### Assessments of the UCLA-MLM can be performed with either Wrapped Runs (historical weather conditions) or Climate Projections.

- #### Details regarding the Wrapped Run approach, the Climate Projections, and the UCLA-MLM Water Budget Model can be found in the UCLA-MLM Report developed for the California State Water Resource Control Board.

- #### Code throughout the notebook is run by hovering over and clicking the  ▶  symbols.


##### -------------------------------------------------------------------------------------------------------------------------------------------------------
##### Creators: Benjamin Bass, Lei Huang, and Sara Graves at UCLA Center for Climate Science
##### Contact Information: benb0228@g.ucla.edu

# General Steps:
### 0. Load data required to run UCLA-MLM
### 1. Define any additional pre-transition export criteria of interest.
### 2. Define any additional post-transition export criteria of interest.
### 3. Define conditions for simulation (e.g. export criteria and emission scenarios to consider).
### 4. Perform simulation (wrapped runs or projections) where projections will include 11 GCMs for each emission scenario selected.
### 5. Optionally write model simulation results to Excel
### 6. Use GUI to create figures from the model simulation.

# 0. Load Data Required to Run UCLA-MLM

##### Upon clicking this first ▶ symbol, you will receive a warning that Google did not create the source code (the UCLA Center for Climate Science team did). You will need to select "Run Anyway" to continue. This is the only time you need to do this.

In [2]:
!pip install distinctipy
import gdown

url = 'https://drive.google.com/uc?id=1ZHEyh8RCECiHxsNsY5peRNOAsSsOJUZ8'
output = 'Mono_Lake_Interface_lib_v4.py'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?id=1451dCTjwq5bieDtaz-LyO0CRR_BhjuZ1'
output = 'Mono_Lake_Area_Storage_Elev.txt'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?id=1Nk6Lmt0DXfv0Lg_XdDxmZOeQbRzrdFe-'
output = 'details_for_model.csv'
gdown.download(url, output, quiet=False)
url = 'https://drive.google.com/uc?id=1Ok4do9MXFGRPJPpjkVVBbPt1aRMjBjbr'
output = 'data.tar'
gdown.download(url, output, quiet=False)

### untar package of data
!tar -xvf data.tar

# from google.colab import drive
# drive.mount('/content/drive')

Downloading...
From (original): https://drive.google.com/uc?id=1ZHEyh8RCECiHxsNsY5peRNOAsSsOJUZ8
From (redirected): https://drive.google.com/uc?id=1ZHEyh8RCECiHxsNsY5peRNOAsSsOJUZ8&confirm=t&uuid=15ae04ff-39a2-4e02-a802-85130b70ff6b
To: /content/Mono_Lake_Interface_lib_v4.py
100%|██████████| 256k/256k [00:00<00:00, 53.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1451dCTjwq5bieDtaz-LyO0CRR_BhjuZ1
To: /content/Mono_Lake_Area_Storage_Elev.txt
100%|██████████| 3.67k/3.67k [00:00<00:00, 11.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Nk6Lmt0DXfv0Lg_XdDxmZOeQbRzrdFe-
To: /content/details_for_model.csv
100%|██████████| 164/164 [00:00<00:00, 460kB/s]
Downloading...
From: https://drive.google.com/uc?id=1Ok4do9MXFGRPJPpjkVVBbPt1aRMjBjbr
To: /content/data.tar
100%|██████████| 4.07M/4.07M [00:00<00:00, 20.2MB/s]

For_Running_Model/
For_Running_Model/Historical_RYT_SRF/
For_Running_Model/Historical_RYT_SRF/MPI-ESM1-2-HR_r1i1p1f1_hist_ssp585.csv
For_Running_Model/Historical_RYT_SRF/GFDL-ESM4_r1i1p1f1_hist_ssp585.csv
For_Running_Model/Historical_RYT_SRF/ACCESS-CM2_r1i1p1f1_hist_ssp370.csv
For_Running_Model/Historical_RYT_SRF/INM-CM5-0_r1i1p1f1_hist_ssp245.csv
For_Running_Model/Historical_RYT_SRF/FGOALS-g3_r1i1p1f1_hist_ssp245.csv
For_Running_Model/Historical_RYT_SRF/EC-Earth3-Veg_r1i1p1f1_hist_ssp370.csv
For_Running_Model/Historical_RYT_SRF/MIROC6_r1i1p1f1_hist_ssp585.csv
For_Running_Model/Historical_RYT_SRF/KACE-1-0-G_r1i1p1f1_hist_ssp245.csv
For_Running_Model/Historical_RYT_SRF/KACE-1-0-G_r1i1p1f1_hist_ssp370.csv
For_Running_Model/Historical_RYT_SRF/CNRM-ESM2-1_r1i1p1f2_hist_ssp585.csv
For_Running_Model/Historical_RYT_SRF/EC-Earth3-Veg_r1i1p1f1_hist_ssp245.csv
For_Running_Model/Historical_RYT_SRF/IPSL-CM6A-LR_r1i1p1f1_hist_ssp585.csv
For_Running_Model/Historical_RYT_SRF/FGOALS-g3_r1i1p1f1_hist_s

In [ ]:
### Load Python Libraries

# Adjusting the HTML to widen the Jupyter notebook view
from IPython.display import display, HTML  # Updated import statement

import Mono_Lake_Interface_lib_v4 as mlib
import numpy as np

import ipywidgets as widgets
from ipywidgets import interact, interactive, interact_manual, fixed
from IPython.display import display, clear_output

# Set the width to 100% for maximum width
display(HTML("<style>.container { width:80% !important; }</style>"))

import warnings
warnings.filterwarnings("ignore")

# # ########## IF NEED TO RELOAD PYTHON FILE ########
# import importlib
# import Mono_Lake_Interface_lib_v3 as mlib
# importlib.reload(mlib)  # Reload the module


#### File Paths
#### Necessary File Paths
GCM_Data_Path = './For_Running_Model'
ERA5_Data = f'{GCM_Data_Path}/ERA5_Historical_Data'
Model_file = './details_for_model.csv'
Lake_file = './Mono_Lake_Area_Storage_Elev.txt'
#(GCM_Data_Path, Model_file, Lake_file) = mlib.get_data_path()
ERA5_Data = f'{GCM_Data_Path}/ERA5_Historical_Data'

### Load GCM Names, Pre-Defined Pre- and Post-Transition Policies

### Load GCMs That Are Available
(simple_GCM_Name_List, GCM_Member_List) = mlib.get_gcms(GCM_Data_Path)

### Load Pre-Defined Policies
Policy_Data = mlib.create_dwp_policy()

### Set Up Post-Transition Dictionary Options
Post_Transition_Policy_Data = {}
Post_Transition_Policy_Data['D_1631'] = Policy_Data['A8']
Post_Transition_Policy_Data['None'] = []

#  Forcing Data and Model Code (only relevant for those interested in details):

  - After running "0. Load Data Required to Run UCLA-MLM" click the 📁 icon to the left. This will display files relevant to the water budget code and forcing data.

  - Forcing Data: The wrapped run and climate projection atmospheric and flow data can be viewed by  and then "For_Running_Model". Inside this folder, the historical reconstruction of observed weather conditions, which is used for the wrapped runs, can be found in "ERA5_Historical_Data". The climate model data can be found in "Dynamic_RYT_SEF". Note, each climate model and emission scenario has its own file in "Dynamic_RYT_SEF". The actual water levels are determined later when the water budget model is simulated against the export criteria that are of interest (you must follow steps below to get the water level).

  - Model Code: This notebook heavily relies on the code in the python file called "Mono_Lake_Interface_lib_v4.py".
    - Within this file, a function called `predict_Mono_Lake_Water_Level_Added_Policies` contains the water budget model code.
    - The creation of policies and figures depend on a variety of functions within the python file. If comfortable with python, you can add flexibility to the policy and figure creation.

# List Export Criteria that are pre-loaded
  - A given export criteria includes pre- and post-transition conditions.
  - Post-transition conditions begin when a water level of 6,391 feet is reached.
  - Pre-loaded pre-transition export criteria include Existing Policy (A1), No Exports (A2), and additional export criteria (A3-A10) that were developed by the Mono Lake Technical Group and are outlined in the UCLA-MLM Report.
  - Pre-loaded post-transition export criteria include Existing Policy (D-1631) and 'None' which uses the pre-transition export criteria at the time 6,391 ft is reached.
  - Additional pre-transition (step 1) and post-transition (step 2) export criteria can be created in this notebook.
  - Note, policy is used interchangeablly with export criteria; however, the only actual policy currently in effect is the A1 pre-transition policy with the D-1631 post-transition policy (A1+D-1631).

In [ ]:
print(f'\nThese Pre-Transition Export Criteria Are Currently Defined: {list(Policy_Data.keys())}')
print(f'\nThese Post-Transition Export Criteria Are Currently Defined: {list(Post_Transition_Policy_Data.keys())}')
print('\nBelow Can Create Additional Pre- and Post-Transition Export Criteria')


These Pre-Transition Export Criteria Are Currently Defined: ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10']

These Post-Transition Export Criteria Are Currently Defined: ['D_1631', 'None']

Below Can Create Additional Pre- and Post-Transition Export Criteria


# 1. Option to Create Additional Pre-Transition Export Criteria

  - Insert `Number of phases` and `Name of policy`
    - `Number of phases` = number of unique phases you want to define for a given export criteria.
  - Then click `1) Define Policy`. This will update and visualize a table that needs to be filled out. The table characteristics reflect the `Number of phases`. For each phase, need to insert export amounts allowed and corresponding water levels.
    - Note, the water level thresholds and corresponding export amounts in the table should be space seperated. Water level thresholds should be in ascending order (lower to higher water levels).
    - If export criteria includes more than one phase, then you must define the `Dyn_Type` and `Phase_Thresh`.
      - `Dyn_Type` = year or level
        - year indicates that after x years a shift in phase will occur.
        - level indicates that the phase will shift when a water level is reached.
      - `Phase_Thresh` = where the # of years or water level (ft) needs to be defined to trigger a year or level phase shift.

  - Click `2) Review Policy` to visualize the policy you defined
    - If want to see an example export criteria, simply enter 1 for `Number of phases`, and then `2) Review Policy`
    - After clicking `2) Review Policy` you can click `1) Define Policy` again if you want to further update the export criteria

  - If policy you have defined is agreeable upon review, add to list of policies that can be evaluated by clicking `3) Add Policy`
    - Note, this does not mean the policy will be evaluated, but that you can evaluate it if you would like to when you perform the simulation in Step 4.

In [ ]:
mlib.create_user_policy(Policy_Data)

### At any point, you can run the line below to display a list of pre-transition export criteria that have been defined.

In [ ]:
print(f'Pre-Transition Export Criteria: {list(Policy_Data.keys())}')

Pre-Transition Export Criteria: ['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9', 'A10', 'U1']


# Overview of Transition Export Criteria and Post-Transition Export Criteria

![Alt text](images\Table%204-1%20Overview%20of%20Transition%20and%20Post-Transition%20Export%20Criteria.png)

## Transition Export Criteria
![Alt text](images/Transition%20Export%20Criteria%20A1%20to%20A8.png)
![Alt text](images/Transition%20Export%20Criteria%20A9%20to%20A10.png)

# 2. Option to Create Additional Post-Transition Export Criteria

  - Recall this is the policy that is implimented after 6,391 feet is reached.
  - Note, the post-transition policy is limited to a single phase.

In [ ]:
mlib.create_user_PT_policy(Post_Transition_Policy_Data)

### Display List of post-transition export criteria that have been defined

In [ ]:
print(f'Post-Transition Export Criteria: {list(Post_Transition_Policy_Data.keys())}')

Post-Transition Export Criteria: ['D_1631', 'None', 'U2']


## Post-Transition Export Criteria

![Alt text](images/Post-Transition%20Export%20Criteria.png)

# 3. Select Options for Actual Model Run
  - This will always run all Global Climate Models (11 GCMs) if Projections is chosen.
  - This will always run all pre-transition export criteria selected with all the post-transition export criteria selected.

### Overview of Shared Socioecononomic Pathways (SSPs)
CMIP6 simulates GCMs under historical emissions and different greenhouse gas emission trajectories, known as Shared Socioeconomic Pathways (SSPs). Different SSPs exist to allow for assessments of how varying levels of emissions related to different societal choices may influence future climate change.

- SSP2-4.5 includes moderate efforts to mitigate climate change and reflects a world with gradual progress toward sustainability.
- SSP3-7.0 is often considered an intermediate-high or "business as usual" pathway, where global efforts to reduce greenhouse gas emissions are limited.
- SSP5-8.5 is often referred to as the "worst-case scenario" and assumes rapid economic growth fueled by intensive fossil fuel use. 

In [ ]:
### Must Re-Run if Create New Policies Above and Want to Be Able to Select Them Below!
state = mlib.define_model_conditions_interface(Policy_Data, Post_Transition_Policy_Data)

# 4. Run the Water Budget Model

In [ ]:
Policy_Outputs = mlib.get_policy_output(state, Model_file, Lake_file, ERA5_Data, GCM_Data_Path, Policy_Data, Post_Transition_Policy_Data,
                                        GCM_Member_List, simple_GCM_Name_List)

### Additional Information Needed for Creating Figures
key_word = state["key_word"]
Initial_Water_Level = state["Initial_Water_Level"]
Wrapped_or_Projections = state["Wrapped_or_Projections"]
Policy_list = state["Pre_Transition_Policies"]
Post_Transition_Policy_List = state["Post_Transition_Policies"]
SSPs_of_Interest = state["SSPs_of_Interest"]
Start_Year = state["Start_Year"]
End_Year = state["End_Year"]

For 4 Policy Combinations, Will Require Roughly 0.2 Minutes

Projections
Running 2 Pre-Transition Policies with 2 Post-Transition Policies
Policy: A1 + D_1631
Policy: A1 + None
Policy: A2 + D_1631
Policy: A2 + None
Projections done!


# 5. Write Simulated Results to an Excel File
### Data Written Includes: Storage, Exports, and Water Level for each Sequence (wrapped runs or GCM/SSP) and Export Criteria

- After writing, you can view and download the Excel file by clicking 📁 on the left

In [ ]:
#### WRITES EACH DATAFRAME AS A SHEET IN ONE SINGLE EXCEL FILE
mlib.write_data_to_excel(state,Policy_Outputs)

NameError: name 'mlib' is not defined

# 6. Create Figures from Simulation

- You can work with the output Excel file to produce any type of figure.

- However, below there are a variety of additional figures that can rapidly be created based on the simulated results.

- Each figure is interactive, and can display more or less information depending on your choices.

- Clicking `Run Interact` will update and generate the figures. Below `Run Interact` is the location of the source code used to generate the figure (this can be ignored).

- We use the term "Sequence" to refer to an individual wrapped run or climate model projection.

# Load information required to create figures

In [ ]:
### Need to run this to set up GUI for Figures

(Policy_of_Interest, PT_Policy_of_Interest, Variable, SSP_of_Interest, Sequence_of_Interest) = mlib.create_widgets(state, Policy_Outputs)

#### Multiple Choice (Checkbox)
policy_selector_widget, get_selected_policies = mlib.create_checkbox_selector(options=Policy_list,title_text="Pre-Transition Policies")

# Create the selector
pt_policy_selector_widget, get_selected_pt_policies = mlib.create_checkbox_selector(options=Post_Transition_Policy_List,
    title_text="Post-Transition Policies")

variable_selector_widget, get_selected_variables = mlib.create_checkbox_selector(options=['Water_Level','Storage','Exports'],
    title_text="Variables")

### Required for figure to show dry and wet year exports
SSPs_of_Interest = ['ssp245','ssp370','ssp585']
ryt_sef_xarray = mlib.get_runoff_info(Wrapped_or_Projections, ERA5_Data, GCM_Data_Path, GCM_Member_List,
                                      SSPs_of_Interest,key_word, Start_Year, End_Year)

# Figures

<span style="font-size:22px;">Figure: Results for Given Export Criteria and All Sequences</span><br>
<span style="font-size:16px;">Select Single Export Criteria, Variable of Interest, SSP if Projections</span>

In [ ]:
#### PLOT EVERY GCM OR SEQUENCE FOR A GIVEN PRE+POST POLICY (AND SSP IF GCM)
#display(HTML(f'<p style="font-size:24px;">Plot results for every {key_word} (select single pre- and post-transition policy)</p>'))

if Wrapped_or_Projections == 'Projections':
    ssp_of_interest = SSP_of_Interest
else:
    ssp_of_interest = fixed('')
interact_manual(mlib.plot_variable_time_series_by_model,
                Wrapped_or_Projections=fixed(Wrapped_or_Projections),
                Policy_of_Interest=Policy_of_Interest,
                PT_Policy_of_Interest=PT_Policy_of_Interest,
                Variable=Variable,
                SSP_of_Interest=ssp_of_interest,
                Policy_Outputs=fixed(Policy_Outputs),
                key_word=fixed(key_word),
                Start_Year=fixed(Start_Year),
                End_Year=fixed(End_Year),
                Sequence_of_Interest = Sequence_of_Interest)

interactive(children=(Select(description='Policy of Interest:', options=('A1', 'A2'), rows=2, style=Descriptio…

<function Mono_Lake_Interface_lib_v4.plot_variable_time_series_by_model(Wrapped_or_Projections, Policy_of_Interest, PT_Policy_of_Interest, Variable, SSP_of_Interest, Policy_Outputs, key_word, Start_Year, End_Year)>

<span style="font-size:22px;">Figure: Plot results for a single Sequence and multiple pre- and post-transition export criteria</span><br>

In [ ]:
### PLOT RESULTS FOR EACH INDIVIDUAL GCM OR SEQUENCE FOR SEVERAL POLICIES

#display(HTML(f'<p style="font-size:24px;">Plot results for a single {key_word} (can select multiple pre- and post-transition policies)</p>'))

display(policy_selector_widget)
display(pt_policy_selector_widget)

# Use the wrapper with interact_manual
if Wrapped_or_Projections == 'Projections':
    ssp_of_interest = SSP_of_Interest
else:
    ssp_of_interest = fixed('')

interact_manual(mlib.wrapper_plot_variable_time_series,
                Wrapped_or_Projections=fixed(Wrapped_or_Projections),
                mlib_plot_function=fixed(mlib.plot_variable_time_series_by_policy),
                get_selected_policies=fixed(get_selected_policies),
                get_selected_pt_policies=fixed(get_selected_pt_policies),
                Variable=fixed('Water_Level'),
                SSP_of_Interest=SSP_of_Interest,
                Policy_Outputs=fixed(Policy_Outputs),
                key_word=fixed(key_word),
                Start_Year=fixed(Start_Year),
                End_Year=fixed(End_Year),
                Sequence_of_Interest=Sequence_of_Interest)

interactive(children=(Select(description='SSP of Interest:', index=1, options=(np.str_('ssp245'), np.str_('ssp…

<function Mono_Lake_Interface_lib_v4.wrapper_plot_variable_time_series(Wrapped_or_Projections, mlib_plot_function, get_selected_policies, get_selected_pt_policies, **kwargs)>

<span style="font-size:22px;">Figure: Plot ensemble mean of Sequences for multiple pre- and post-transition export criteria</span><br>
<span style="font-size:16px;">You can also include 1 standard deviation and/or min/max water level from all sequences</span>

In [ ]:
#### PLOT THE ENSEMBLE MEAN, STDEV, MIN/MAX FROM THE DIFFERENT GCMS OR SEQUENCES
# Create widgets for standard deviation and min-max options
Stdev_widget = widgets.Checkbox(
    value=False,
    description='Include standard deviation',
    style={'description_width': 'initial'},
    disabled=False)

Min_Max_widget = widgets.Checkbox(
    value=False,
    description='Include min and max',
    style={'description_width': 'initial'},
    disabled=False)

# Display the checkbox widgets for pre- and post-transition policies
display(policy_selector_widget)
display(pt_policy_selector_widget)

# Use interact_manual with the updated wrapper function
interact_manual(
    mlib.wrapper_plot_variable_time_series_by_policy_mean,
    Wrapped_or_Projections=fixed(Wrapped_or_Projections),
    get_selected_policies=fixed(get_selected_policies),  # Pass the callable function for pre-transition policies
    Variable=Variable,  # Keep Variable unchanged as passed from your code
    SSP_of_Interest=SSP_of_Interest,  # Keep SSP_of_Interest unchanged
    Policy_Outputs=fixed(Policy_Outputs),
    key_word=fixed(key_word),
    Start_Year=fixed(Start_Year),
    End_Year=fixed(End_Year),
    Stdev=Stdev_widget,
    Min_Max=Min_Max_widget,
    get_selected_pt_policies=fixed(get_selected_pt_policies)  # Pass the callable function for post-transition policies
)

interactive(children=(Select(description='Variable of Interest:', options=('Water_Level', 'Storage', 'Exports'…

<function Mono_Lake_Interface_lib_v4.wrapper_plot_variable_time_series_by_policy_mean(Wrapped_or_Projections, get_selected_policies, Variable, SSP_of_Interest, Policy_Outputs, key_word, Start_Year, End_Year, Stdev, Min_Max, get_selected_pt_policies)>

<span style="font-size:20px;">Figure: Boxplot of Results for a given year from all sequences</span><br>
<span style="font-size:16px;">See legend for a description of the boxplot.</span>

In [ ]:
### Boxplot of different GCM/Sequence Water Levels for a given year
display(policy_selector_widget)
display(pt_policy_selector_widget)

# Use interact_manual with the updated wrapper function
interact_manual(
    mlib.wrapper_boxplot_variable_by_policy,
    Wrapped_or_Projections=fixed(Wrapped_or_Projections),
    get_selected_policies=fixed(get_selected_policies),  # Pass the callable function for pre-transition policies
    Variable=Variable,  # Dropdown for variables
    SSP_of_Interest=SSP_of_Interest,  # SSP selection as in your original code
    Policy_Outputs=fixed(Policy_Outputs),
    year_of_interest=mlib.get_year_of_interest(state),  # Year of interest widget
    get_selected_pt_policies=fixed(get_selected_pt_policies)  # Pass the callable function for post-transition policies
)

interactive(children=(Select(description='Variable of Interest:', options=('Water_Level', 'Storage', 'Exports'…

<function Mono_Lake_Interface_lib_v4.wrapper_boxplot_variable_by_policy(Wrapped_or_Projections, get_selected_policies, Variable, SSP_of_Interest, Policy_Outputs, year_of_interest, get_selected_pt_policies)>

<span style="font-size:20px;">Figure: Similar to Boxplot above, but separated by post-transition policies.</span><br>

In [ ]:
# Display the checkbox widgets for pre- and post-transition policies and variables
display(policy_selector_widget)  # Pre-transition policy selector
display(pt_policy_selector_widget)  # Post-transition policy selector
display(variable_selector_widget)  # Variable selector (checkbox)

# Use interact_manual with the updated wrapper function
print('Must Select at least two Post-Transition Policies and two Variables to Compare')
interact_manual(
    mlib.wrapper_boxplot_variables_by_policies,
    Wrapped_or_Projections=fixed(Wrapped_or_Projections),
    get_selected_policies=fixed(get_selected_policies),  # Pass the callable function for pre-transition policies
    get_selected_variables=fixed(get_selected_variables),  # Pass the callable function for variables
    SSP_of_Interest=SSP_of_Interest,  # SSP selection as in your original code
    Policy_Outputs=fixed(Policy_Outputs),
    year_of_interest=mlib.get_year_of_interest(state),  # Year of interest widget
    get_selected_pt_policies=fixed(get_selected_pt_policies),  # Pass the callable function for post-transition policies
    key_word=fixed(key_word),
    Start_Year=fixed(Start_Year),
    End_Year=fixed(End_Year)
)

Must Select at least two Post-Transition Policies and two Variables to Compare


interactive(children=(Select(description='SSP of Interest:', index=1, options=(np.str_('ssp245'), np.str_('ssp…

<function Mono_Lake_Interface_lib_v4.wrapper_boxplot_variables_by_policies(Wrapped_or_Projections, get_selected_policies, get_selected_variables, SSP_of_Interest, Policy_Outputs, year_of_interest, get_selected_pt_policies, key_word, Start_Year, End_Year)>

<span style="font-size:20px;">Figure: Percent of Sequences above Water Level across simulated time period, separated by post-transition export criteria</span><br>

In [ ]:
### PLOT PERCENT OF GCMS OR SEQUENCES ABOVE WATER LEVEL
#display(HTML(f'<p style="font-size:24px;">Percent of all {key_word}s Above Water Level of Interest, Seperated by Post-Transition Policies</p>'))

# Display the checkbox widgets for pre- and post-transition policies
display(policy_selector_widget)  # Pre-transition policy selector
display(pt_policy_selector_widget)  # Post-transition policy selector

# Use interact_manual with the updated wrapper function
interact_manual(
    mlib.wrapper_plot_gcm_percent_above_level_by_policy,
    Wrapped_or_Projections=fixed(Wrapped_or_Projections),
    get_selected_policies=fixed(get_selected_policies),  # Pass the callable function for pre-transition policies
    get_selected_pt_policies=fixed(get_selected_pt_policies),  # Pass the callable function for post-transition policies
    Variable=fixed('Water_Level'),  # Variable is fixed to 'Water_Level'
    Water_Level_of_Interest=mlib.get_water_level_of_interest(),  # Water level of interest widget
    SSP_of_Interest=SSP_of_Interest,  # SSP selection as in your original code
    Policy_Outputs=fixed(Policy_Outputs),
    key_word=fixed(key_word),
    Start_Year=fixed(Start_Year),
    End_Year=fixed(End_Year)
)

interactive(children=(IntText(value=6391, continuous_update=True, description='Water Level of Interest:', styl…

<function Mono_Lake_Interface_lib_v4.wrapper_plot_gcm_percent_above_level_by_policy(Wrapped_or_Projections, get_selected_policies, get_selected_pt_policies, Variable, Water_Level_of_Interest, SSP_of_Interest, Policy_Outputs, key_word, Start_Year, End_Year)>

<span style="font-size:20px;">Figure: Percent of Sequences that have reached Post-Transition Phase (occurs once 6,391 feet is reached)</span><br>

<span style="font-size:20px;">Note: Only the pre-transition export criteria influences this figure</span><br>

In [ ]:
### PLOT PERCENT OF GCMS OR SEQUENCES IN Post-Transition Phase
#display(HTML(f'<p style="font-size:24px;">Percent of {key_word}s Reached Post-Transition Phase</p>'))

# Display the checkbox widgets for pre- and post-transition policies
display(policy_selector_widget)  # Pre-transition policy selector

# Use interact_manual with the updated wrapper function
interact_manual(
    mlib.wrapper_plot_plot_percent_in_transition_phase,
    Wrapped_or_Projections=fixed(Wrapped_or_Projections),
    get_selected_policies=fixed(get_selected_policies),  # Pass the callable function for pre-transition policies
    SSPs_of_Interest=fixed(SSPs_of_Interest),  # SSP selection as in your original code
    Post_Transition_Policy_List=fixed(Post_Transition_Policy_List),
    Policy_Outputs=fixed(Policy_Outputs),
    Start_Year=fixed(Start_Year),
    End_Year=fixed(End_Year)
)

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…

<function Mono_Lake_Interface_lib_v4.wrapper_plot_plot_percent_in_transition_phase(Wrapped_or_Projections, get_selected_policies, SSPs_of_Interest, Post_Transition_Policy_List, Policy_Outputs, Start_Year, End_Year)>

# The Remainder of figures are only available for Projections (not wrapped runs).

<span style="font-size:20px;">Figure: Percent of All GCMs above a given water level for each SSP</span><br>

<span style="font-size:20px;">Note: You can select many pre-transition export criteria, but only one post-transition export criteria</span><br>

In [ ]:
### For a single post-transition policy, plot number of GCMs above water level for each ssp (code not relevant for wrapped runs)
#display(HTML(f'<p style="font-size:24px;">Percent of all {key_word}s Above Water Level of Interest (single post-transition phase) Seperated by SSPs</p>'))

# Display the checkbox widgets for pre- and post-transition policies
display(policy_selector_widget)  # Pre-transition policy selector

# Use interact_manual with the updated wrapper function
interact_manual(
    mlib.wrapper_plot_gcm_percent_above_level_by_ssp,
    Wrapped_or_Projections=fixed(Wrapped_or_Projections),
    get_selected_policies=fixed(get_selected_policies),  # Pass the callable function for pre-transition policies
    Selected_PT_Policy=PT_Policy_of_Interest,  # Get post-transition policies
    ssps_to_plot=fixed(SSPs_of_Interest),  # Fixed SSP selection
    Variable=fixed('Water_Level'),  # Variable is fixed to 'Water_Level'
    Water_Level_of_Interest=mlib.get_water_level_of_interest(),  # Water level of interest widget
    Policy_Outputs=fixed(Policy_Outputs),
    key_word=fixed(key_word),
    Start_Year=fixed(Start_Year),
    End_Year=fixed(End_Year)
)

interactive(children=(Select(description='Post Transition Policy of Interest:', options=('D_1631', 'None'), ro…

<function Mono_Lake_Interface_lib_v4.wrapper_plot_gcm_percent_above_level_by_ssp(Wrapped_or_Projections, get_selected_policies, Selected_PT_Policy, ssps_to_plot, Variable, Water_Level_of_Interest, Policy_Outputs, key_word, Start_Year, End_Year)>

<span style="font-size:20px;">Figure: Difference in Likelihood above a User-Defined Water Level and Difference in Exports for Each Export Criteria Compared Against Existing Export Criteria or No Exports</span><br>
<span style="font-size:16px;">The top left figure shows the export policy being compared against, all other figures show difference for each decade (labeled above each bar) from the start of simulation</span>

In [ ]:
# Display the pre-transition policy selector widget
A1_or_A2 = widgets.Select(
        options = ['A1','A2'],
        value = 'A1',
        description = 'Compare to Existing Export Criteria (A1+D-1631) or No Exports (A2+None):',
        style = {'description_width': 'initial'},
        rows = 1,
        continuous_update = True,
        disabled = False)

# Use interact_manual with the updated wrapper function
interact_manual(
    mlib.plot_percent_diff_pre_post_trans_with_export,
    Policy_Outputs=fixed(Policy_Outputs),
    ssps_to_plot=fixed(SSPs_of_Interest),
    Water_Level_of_Interest=mlib.get_water_level_of_interest(),
    policies_to_plot=fixed(Policy_list),
    pt_policy=PT_Policy_of_Interest,
    Variable=fixed('Water_Level'),
    Exports_Variable=fixed('Exports'),
    Compare_to_A1_or_A2=A1_or_A2,
    Start_Year=fixed(Start_Year),
    End_Year=fixed(End_Year),
    ryt_sef_xarray=fixed(ryt_sef_xarray)
)

interactive(children=(IntText(value=6391, continuous_update=True, description='Water Level of Interest:', styl…

<function Mono_Lake_Interface_lib_v4.plot_percent_diff_pre_post_trans_with_export(Policy_Outputs, ssps_to_plot, Water_Level_of_Interest, policies_to_plot, pt_policy, Variable, Exports_Variable, Compare_to_A1_or_A2, Start_Year, End_Year, ryt_sef_xarray)>

<span style="font-size:20px;">Figure: How the likelihood above a User-Defined Water Level Compares to No Exports (A2+None)</span><br>
<span style="font-size:16px;">The top left plot shows the likelihood of being above water level for No Exports for each decade (decades labeled above each bar) from start of simulation, and all other figures show average difference from No Exports</span>

In [ ]:
# Use interact_manual with the updated wrapper function
interact_manual(
    mlib.plot_percent_diff_pre_post_trans_without_export,
    Policy_Outputs=fixed(Policy_Outputs),
    ssps_to_plot=fixed(SSPs_of_Interest),
    Water_Level_of_Interest=mlib.get_water_level_of_interest(),
    policies_to_plot=fixed(Policy_list),
    pt_policy=PT_Policy_of_Interest,
    Variable=fixed('Water_Level'),
    Compare_to_A1_or_A2=fixed('A2'),
    Start_Year=fixed(Start_Year),
    End_Year=fixed(End_Year)
)

interactive(children=(IntText(value=6391, continuous_update=True, description='Water Level of Interest:', styl…

<function Mono_Lake_Interface_lib_v4.plot_percent_diff_pre_post_trans_without_export(Policy_Outputs, ssps_to_plot, Water_Level_of_Interest, policies_to_plot, pt_policy, Variable, Compare_to_A1_or_A2, Start_Year, End_Year)>

<span style="font-size:20px;">Figure: How likelihood above different water levels compares to No Exports (A2+None) and the Average Dry and Wet Exports for a Given Range of Years</span><br>

In [ ]:
Start_Year_Index = widgets.IntText(
            value=25,
            description=f'Index of Start Year:',
            style={'description_width': 'initial'},
            continuous_update=True,
            disabled=False)
End_Year_Index = widgets.IntText(
            value=35,
            description=f'Index of End Year:',
            style={'description_width': 'initial'},
            continuous_update=True,
            disabled=False)

# Use interact_manual with the updated wrapper function
display(policy_selector_widget)
display(pt_policy_selector_widget)
interact_manual(
    mlib.line_plot_policy_performance,
    Policy_Outputs=fixed(Policy_Outputs),
    start_year_index=Start_Year_Index,
    end_year_index=End_Year_Index,
    get_policies_to_plot=fixed(get_selected_policies),
    ssps_to_plot=fixed(SSPs_of_Interest),
    get_pt_policies=fixed(get_selected_pt_policies),
    water_levels_of_interest_str=mlib.get_water_levels_of_interest(),
    Start_Year=fixed(Start_Year),
    ryt_sef_xarray=fixed(ryt_sef_xarray)
)

interactive(children=(IntText(value=25, continuous_update=True, description='Index of Start Year:', style=Desc…

<function Mono_Lake_Interface_lib_v4.line_plot_policy_performance(Policy_Outputs, start_year_index, end_year_index, get_policies_to_plot, ssps_to_plot, get_pt_policies, water_levels_of_interest_str, Start_Year, ryt_sef_xarray)>

<span style="font-size:20px;">Figure: Heatmap of the likelihood of being above User-Defined Water Levels for a User-Defined Range of Years</span><br>

In [ ]:
Start_Year_Index = widgets.IntText(
            value=45,
            description=f'Index of Start Year:',
            style={'description_width': 'initial'},
            continuous_update=True,
            disabled=False)
End_Year_Index = widgets.IntText(
            value=76,
            description=f'Index of End Year:',
            style={'description_width': 'initial'},
            continuous_update=True,
            disabled=False)

# Use interact_manual with the updated wrapper function
display(policy_selector_widget)
interact_manual(
    mlib.table_plot_policy_performance,
    Policy_Outputs=fixed(Policy_Outputs),
    start_year_index=Start_Year_Index,
    end_year_index=End_Year_Index,
    get_policies_to_plot=fixed(get_selected_policies),
    ssps_to_plot=mlib.get_ssps_of_interest(),
    get_pt_policies=fixed(get_selected_pt_policies),
    water_levels_of_interest_str=mlib.get_water_levels_of_interest(),
    Start_Year=fixed(Start_Year),
    ryt_sef_xarray=fixed(ryt_sef_xarray)
)

interactive(children=(IntText(value=45, continuous_update=True, description='Index of Start Year:', style=Desc…

<function Mono_Lake_Interface_lib_v4.table_plot_policy_performance(Policy_Outputs, start_year_index, end_year_index, get_policies_to_plot, ssps_to_plot, get_pt_policies, water_levels_of_interest_str, Start_Year, ryt_sef_xarray)>

# End of Model Notebook